# 猫狗大战

## 特征提取

In [ ]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import datetime

print('start')
starttime = datetime.datetime.now()

model = Xception(weights='imagenet', include_top=False)

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

generator = datagen.flow_from_directory(
    'dataset-mini/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode=None,
    shuffle=False)
bottleneck_features_train = model.predict_generator(generator, 200)
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = datagen.flow_from_directory(
    'data/validation',
    target_size=(299, 299),
    batch_size=32,
    class_mode=None,
    shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, 100)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

print('complete!')
endtime = datetime.datetime.now()
print (endtime - starttime)

Using TensorFlow backend.


start
Found 200 images belonging to 2 classes.


## 搭建模型

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense

train_data = np.load(open('bottleneck_features_train.npy'))
train_labels = np.array([0] * 100 + [1] * 100)

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 50 + [1] * 50)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          nb_epoch=50, batch_size=32,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')